### Imports and loading data

In [1]:
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

### Inspecting data

### Dropping redundant columns

### Cleaning and merging negative and positive reviews

### Lower case

### Remove numbers

### Remove punctuation

### Remove stopwords

### Lemmatizing

### Custom scaling and rounding

### Exporting to csv

------------------------------------- NEW FILE --------------------------

In [2]:
import numpy as np
import string
import nltk
from nltk.tokenize import word_tokenize

# Import CSV

In [3]:
data = pd.read_csv("../raw_data/clean_dataset_1.csv", index_col=0)
data

,reviews,review_score
0,angry made post available via possible site us...,0.29
1,real complaint hotel great great location surr...,0.75
2,room nice elderly bit difficult room two story...,0.71
3,room dirty afraid walk barefoot floor looked c...,0.38
4,booked company line showed picture room though...,0.67
...,...,...
515733,trolly staff help take luggage room location,0.70
515734,hotel look like surely breakfast ok got earlie...,0.58
515735,ac useless hot week vienna gave hot air,0.25
515736,room enormous really comfortable believe famil...,0.88


# Remove less than 10 words (To Do)

In [4]:
data['length'] = data['reviews'].apply(lambda x: len(word_tokenize(str(x))))
data.drop(data[data['length'] < 11].index, inplace=True)
data.drop(columns=['length'], inplace=True)
data.reset_index(drop=True, inplace=True)
data

,reviews,review_score
0,angry made post available via possible site us...,0.29
1,real complaint hotel great great location surr...,0.75
2,room nice elderly bit difficult room two story...,0.71
3,room dirty afraid walk barefoot floor looked c...,0.38
4,booked company line showed picture room though...,0.67
...,...,...
299735,noisy night air causing loud banging heating p...,0.63
299736,put fotos cinde room see booking location give...,0.54
299737,wifi location tea coffee room rather cheap far...,0.58
299738,breakfast excelent lot fress fruit hotel good ...,1.00


# WARNING - TOO MANY DROPS

In [5]:
data.shape

(299740, 2)

# Remove Words with numbers (To Do - DONT DO - Checked with BEN)

In [6]:
# def remove_words_w_numbers(text):
#     # Tokenizing
#     text_tokenized = word_tokenize(text)
#     text = [str(word) for word in text_tokenized] # if not word.isdigit()] 
#     text = [list(word) for word in text] # if not letter.isdigit()]
# #     text = ' '.join([word for word in text])
# #     text = ' '.join([word for word in text])
#     return text

In [7]:
# remove_words_w_numbers(str(data['reviews'][4] + " 3rd"))

# Vectorizer: Bag of words - Model: LinearRegression

# Vectorizer: Tf-Idf - Model: LinearRegression (max_features=50) (0.21)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer(max_features=50)

In [26]:
X = tf_idf_vectorizer.fit_transform(data['reviews'][:10000])
y = data['review_score'][:10000]

In [27]:
from sklearn.model_selection import train_test_split

# Hold out 
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X.toarray(),columns = tf_idf_vectorizer.get_feature_names()), y, test_size=0.2)

#check
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 50), (2000, 50), (8000,), (2000,))

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate

model = LinearRegression()
model.fit(X_train,y_train)

result = cross_validate(model,X_test,y_test)

result['test_score'].mean()

0.2087041806052611

# loop - max_features=50  (pipiline)  - grid search (50, 100 , 200)  / try different alphas

# Vectorizer: Tf-Idf - Model: LinearRegression (max_features=200) (0.38)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer(max_features=200)

In [30]:
X = tf_idf_vectorizer.fit_transform(data['reviews'][:10000])
y = data['review_score'][:10000]

In [31]:
from sklearn.model_selection import train_test_split

# Hold out 
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X.toarray(),columns = tf_idf_vectorizer.get_feature_names()), y, test_size=0.2)

#check
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 200), (2000, 200), (8000,), (2000,))

In [61]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate

model = LinearRegression()
model.fit(X_train,y_train)

result = cross_validate(model,X_test,y_test)

result['test_score'].mean()

0.37104277075166936

# Vectorizer: Tf-Idf (max_features=200) - Model: Lasso (-0.01)

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Lasso

model = Lasso()
model.fit(X_train,y_train)

result = cross_validate(model,X_test,y_test)

result['test_score'].mean()

-0.0033445541332157713

# Vectorizer: Tf-Idf - Model: Random Forest (0.31)

#### (0.31)

In [36]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

model = RandomForestRegressor(n_estimators=100)
model.fit(X_train,y_train)

cv_results = cross_val_score(model, X_test,y_test, cv=5, scoring='r2').mean()
cv_results

array([0.32599855, 0.28929107, 0.37346665, 0.32872847, 0.25845931])

In [37]:
# test score
cv_results.mean()

0.3151888082797575

In [47]:
model = RandomForestRegressor(max_depth=80, min_samples_split=3, min_samples_leaf=2, n_estimators=200)
model.fit(X_train,y_train)

cv_results = cross_val_score(model, X_test,y_test, cv=5, scoring='r2').mean()
cv_results

0.317347755547485

In [48]:
model = RandomForestRegressor(max_depth=100, min_samples_split=20, min_samples_leaf=10, n_estimators=200)
model.fit(X_train,y_train)

cv_results = cross_val_score(model, X_test,y_test, cv=5, scoring='r2').mean()
cv_results

0.2856400462332994

In [ ]:
# import matplotlib.pyplot as plt 
# from sklearn.model_selection import learning_curve
# #Learning Curves
# train_sizes = [25,50,75,100,250,500,750,1000,1150]
# # Get train scores (R2), train sizes, and validation scores using `learning_curve`
# train_sizes, train_scores, test_scores = learning_curve(
#     estimator=pipeline_tfidf_l1_lasso, X=data['reviews'], y=y, train_sizes=train_sizes, cv=5)

# # Take the mean of cross-validated train scores and validation scores
# train_scores_mean = np.mean(train_scores, axis=1)
# test_scores_mean = np.mean(test_scores, axis=1)
# plt.plot(train_sizes, train_scores_mean, label = 'Training score')
# plt.plot(train_sizes, test_scores_mean, label = 'Test score')
# #plt.ylabel('r2 score', fontsize = 14)
# #plt.xlabel('Training set size', fontsize = 14)
# plt.title('Learning curves', fontsize = 18, y = 1.03)
# plt.legend()

# Vectorizer: Tf-Idf - Model: KNN (0.13)

In [40]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor()
model.fit(X_train,y_train)

cv_results = cross_val_score(model, X_test,y_test, cv=5, scoring='r2').mean()
cv_results

0.13323591069381152

# Vectorizer: Tf-Idf - SVM - Linear (0.37)

#### Version 1

In [78]:
from sklearn.svm import SVR

model = SVR(kernel='linear')
model.fit(X_train,y_train)

cv_results = cross_val_score(model, X_test,y_test, cv=5, scoring='r2').mean()
cv_results

0.3594117159885309

In [97]:
model = SVR(kernel='linear', gamma=3, C=10)
model.fit(X_train,y_train)

cv_results = cross_val_score(model, X_test,y_test, cv=5, scoring='r2').mean()
cv_results

0.350323186253943

#### Version 2

In [84]:
model = SVR(kernel='poly')
model.fit(X_train,y_train)

cv_results = cross_val_score(model, X_test,y_test, cv=5, scoring='r2').mean()
cv_results

0.2462946823187738

### Version 3

In [85]:
model = SVR(kernel='rbf')
model.fit(X_train,y_train)

cv_results = cross_val_score(model, X_test,y_test, cv=5, scoring='r2').mean()
cv_results

0.3876106519241459

In [109]:
model = SVR(kernel='rbf', gamma=0.15, C=20)
model.fit(X_train,y_train)

cv_results = cross_val_score(model, X_test,y_test, cv=5, scoring='r2').mean()
cv_results

0.3736975954943555

### Version 4

In [86]:
model = SVR(kernel='sigmoid')
model.fit(X_train,y_train)

cv_results = cross_val_score(model, X_test,y_test, cv=5, scoring='r2').mean()
cv_results

-0.24889806043884904

### Version 5 - SVR(kernel='linear') - RandomSearch

### Version 6 - SVR(kernel='rbf') - RandomSearch 

#  20.000 DataPoints and max_features=500

In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer(max_features=500)

X = tf_idf_vectorizer.fit_transform(data['reviews'][:20000])
y = data['review_score'][:20000]

from sklearn.model_selection import train_test_split

# Hold out 
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X.toarray(),columns = tf_idf_vectorizer.get_feature_names()), y, test_size=0.2)

#check
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16000, 500), (4000, 500), (16000,), (4000,))

#### Model: LinearRegression 

In [114]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train,y_train)

result = cross_validate(model,X_test,y_test)

result['test_score'].mean()

0.34630050416534763

#### Model: Random Forest  

In [116]:
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train,y_train)

cv_results = cross_val_score(model, X_test,y_test, cv=5, scoring='r2').mean()
cv_results

0.30810832845252856

#### Model: SVM - Linear

In [115]:
from sklearn.svm import SVR

model = SVR(kernel='linear')
model.fit(X_train,y_train)

cv_results = cross_val_score(model, X_test,y_test, cv=5, scoring='r2').mean()
cv_results

0.3479263037658146

#### Model: SVM - rbf

In [113]:
model = SVR(kernel='rbf')
model.fit(X_train,y_train)

cv_results = cross_val_score(model, X_test,y_test, cv=5, scoring='r2').mean()
cv_results

0.36662833287129154

# Export

In [ ]:
# import pickle

# # Export pipeline as pickle file
# with open("pipeline.pkl", "wb") as file:
#     pickle.dump(best_pipeline, file)